### Imports

In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
from io import StringIO
import json
import os, sys

sys.path.append("../")

In [2]:
from scripts.converter import xml_to_json
from scripts.parser_fucn import find_section_by_optimized_path
from scripts.parser_sex_age import get_sex, get_age
from scripts.parser_gospital import get_gosp_info
from scripts.parser_amnez_disease import get_amnez_d
from scripts.parser_condition import get_condition
from scripts.parser_diagn import get_diagnosis
from scripts.parses_amnez_life import get_amnez_life
from scripts.parser_wards import get_ward_table
from scripts.parse_desease_features import get_features_from_diagnosis_table

### table creation

In [4]:
records = []
json_dir = os.path.join("../", "data", "json_records", "DocsNew1")
for json_file in os.listdir(json_dir):
    path_json = os.path.join(json_dir, json_file)

    # read json
    with open(path_json, "r", encoding="utf-8") as file:
        data = json.load(file)

    data_in_record = {
        "record": json_file.removesuffix(".json"),
        "gender": get_sex(data),
        "birthday": get_age(data),
        "amnez_life": get_amnez_life(data),
        "amnez_disease": get_amnez_d(data),
        "condition": get_condition(data),
    }
    diagnosis_code_and_desc = get_features_from_diagnosis_table(data)
    data_in_record.update(diagnosis_code_and_desc)

    a, b, c = get_gosp_info(data)  # a same as diagnosis_table
    data_in_record.update(
        {
            "gospitalisation_type": b,
            "gospitalisation_delivery": c,
        }
    )

    records.append(data_in_record)

records_in_dataframe = pd.DataFrame(records)

In [5]:
records_in_dataframe

,record,gender,birthday,amnez_life,amnez_disease,condition,complication_of_main_disease_desc,main_disease_desc,secondary_disease_desc,complication_of_main_disease_code,main_disease_code,secondary_disease_code,gospitalisation_type,gospitalisation_delivery,background_disease_desc,background_disease_code
0,EMD_EPIC_DISCHARGE_182985535_182985535,Мужской,19710308,Б,"Считает себя больным с 13.01.2023 г, когда в 1...","[Состояние: Тяжелое, Жалобы: на пекуче-жгучую ...",ОСН 1 по Killip.,ИБС. Острый инфаркт миокарда нижней стенки ЛЖ ...,Ожирение 2 степени (ИМТ 36),[I50.1],[I21.4],[E66.9],Первичный,Плановая госпитализация,NaN,NaN
1,EMD_EPIC_DISCHARGE_186840484_186840484,Женский,19650620,"Эпидемиологический анамнез:Со слов пациента, с...",08.06.2023г около 20.30 в покое появились давя...,"[Состояние: Тяжелое, Жалобы: на давящие боли з...",ОСН I по Killip.,ИБС. Инфаркт миокарда с подъёмом сегмента ST п...,Ожирение 1 степени (ИМТ 32),[I50.1],[I21.0],[E66.9],Первичный,Доставлен бригадой скорой помощи,Гипертоническая болезнь III стадии. Неконтроли...,[I11.9]
2,EMD_EPIC_DISCHARGE_187352881_187352881,Мужской,19790326,"Эпидемиологический анамнез:Со слов пациента, с...","ранее ИБС отрицает, впервые стал отмечать колю...","[Состояние: Тяжелое, Жалобы: на давящие боли з...",ОСН I по Killip,ИБС. Острый инфаркт миокарда передне-перегород...,Ожирение 2 степени по ВОЗ(ИМТ 36 кг/м2),[I50.1],[I21.0],[E66.9],Повторный,Доставлен бригадой скорой помощи,NaN,NaN
3,EMD_EPIC_DISCHARGE_184390624_184390624,Мужской,19620330,"Эпидемиологический анамнез:\nСо слов пациента,...",Клиника ангинозных болей с 2014г. тогда же выя...,"[Состояние: Средней тяжести, Жалобы: на кратко...",ОСН 1 по Killip.,ИБС. Прогрессирующая стенокардия. АКШ от 17.09...,"Пароксизмальная форма фибрилляции предсердий, ...",[I50.1],[I20.0],"[I48.0, E66.9, N28.1]",Первичный,Доставлен бригадой скорой помощи,Гипертоническая болезнь III стадии. Неконтроли...,[I11.9]
4,EMD_EPIC_DISCHARGE_187743338_187743338,Мужской,19630407,"Эпидемиологический анамнез:Со слов пациента, с...","считает себя больным с 2021г, когда впервые в ...","[Состояние: Тяжелое, Жалобы: на давящие, жгучи...",ОСН I по Killip,ИБС. Острый инфаркт миокарда передне-перегород...,"Атеросклероз БЦС, гемодинамически незначимый, ...",[I50.1],[I21.0],"[E66.9, E74.8, I70.8]",Первичный,Плановая госпитализация,NaN,NaN
5,EMD_EPIC_DISCHARGE_183327505_183327505,Мужской,19570115,Б,"Считает себя больным с 18.01.2023г. с 11:00ч.,...","[Состояние: Средней тяжести, Жалобы: На давящи...",ОСН I по Killip.,"ИБС. Инфаркт миокарда передне-перегородочной, ...",Ожирение 3 степени (ИМТ 41 кг/м2)Персистирующа...,[I50.1],[I21.4],"[I48.0, E66.9]",Первичный,Плановая госпитализация,Гипертоническая болезнь III стадии. Неконтроли...,[I11.9]
6,EMD_EPIC_DISCHARGE_187581845_187581845,Женский,19680630,"Эпидемиологический анамнез:Со слов пациента, с...","Считает себя больной с 03.07.2023г., когда пос...","[Состояние: Средней тяжести, Жалобы: на давящи...",ОСН по Killip I. Ранняя постинфарктная стенока...,"ИБС. ОИМ передне-перегородочной, верхушечной с...",Поливалентная аллергия.Тиреотоксикоз неуточненный,[I50.1],[I21.0],"[E05.9, T78.4]",Повторный,Плановая госпитализация,Гипертоническая болезнь III стадии. Неконтроли...,[I11.9]
7,EMD_EPIC_DISCHARGE_187732178_187732178,Мужской,19660428,Общебиографические сведения на севере длитель...,Повышение АД много лет. Максимальное АД 200/ м...,"[Состояние: Средней тяжести, Жалобы: на боль н...",ХСН 1 с сохраненной ФВ. ФК 3. Астенический си...,ИБС. Острый инфаркт миокарда нижне-боковой сте...,"Атеросклероз артерий нижних конечностей, значи...",[I50.1],[I21.4],"[K25.6, K29.6, I70.2, I70.8]",Повторный,Плановая госпитализация,Гипертоническая болезнь III стадия. Медикамент...,[I11.9]
8,EMD_EPIC_DISCHARGE_188329516_188329516,Мужской,19630430,"Эпидемиологический анамнез:Со слов пациента, с...",ИБС с 2007 года когда перенес ИМ. Проходил ста...,"[Состояние: Тяжелое, Жалобы: на жгучие боли за...",ОСН 1 по Killip.,ИБС. Острый инфаркт миокарда по 